### Imports

In [2]:
import numpy as np
import random
# import to do training
from tpg.trainer import Trainer
# import to run an agent (always needed)
from tpg.agent import Agent
# faster training
import multiprocessing as mp

### Grid World

In [18]:
# based (for now) on Downing's Figure 4.1
class GridWorldv1:
    def __init__(self, rows, cols, win_state, starting_state):  
        self.rows = rows
        self.cols = cols
        self.starting_state = starting_state
        self.win_state = win_state
        self.current_state = self.starting_state

    def sample_action (self):
        rand = random.uniform(0, 1)
        if (rand >= 0) and (rand < 0.25):
            return 0
        elif (rand >= 0.25) and (rand < 0.5):
            return 1
        elif (rand >= 0.5) and (rand < 0.75):
            return 2
        else:
            return 3
        
    def reset (self):
        self.current_state = self.starting_state
        return self.current_state
        
    def check_win (self):
        if self.current_state == self.win_state:
            return True
        return False

    # just reset for now...
    def close (self):
        self.current_state = self.starting_state
        return 1
    
    def step (self, action):
        # north
        if action == 0:
            next = (self.current_state[0] + 1, self.current_state[1])
        # south
        elif action == 1:
            next = (self.current_state[0] - 1, self.current_state[1])
        # east
        elif action == 2:
            next = (self.current_state[0], self.current_state[1] + 1)
        # west
        else:
            next = (self.current_state[0], self.current_state[1] - 1)

        terminate = False
        reward = 0
        # check if move is legal
        if (next[0] >= 0 and next[0] <= 9) and (next[1] >= 0 and next[1] <= 9):            
            illegal = 0
            for i in range(5):
                if next == (0, i):
                    illegal = 1
                    
            if (illegal == 0):
                self.current_state = next
                reward += 0.01
                #print(self.current_state)
            else:
                #print('illegal move!')
                reward = -1
                terminate = True
        else:
            #print('out of bounds!')
            reward = -1
            terminate = True
        
        if self.check_win():
            reward += 100
            terminate = True
        
        return self.current_state, reward, terminate

### GridWorld Test

In [19]:
env = GridWorldv1(10, 10, (9, 0), (0, 0))
state = env.reset()
#action = agent.act(state)
action = env.sample_action()
state, reward, isDone = env.step(action)
print(state)
print(reward)
print(isDone)

(0, 0)
-1
True


### TPG Agent

In [24]:
def runAgent(args):
    agent = args[0]
    envName = args[1]
    scoreList = args[2]
    numEpisodes = args[3] # number of times to repeat game
    numFrames = args[4] 
    
    # skip if task already done by agent
    if agent.taskDone(envName):
        print('Agent #' + str(agent.agentNum) + ' can skip.')
        scoreList.append((agent.team.id, agent.team.outcomes))
        return
    
    env = GridWorldv1(10, 10, (9, 0), (0, 0))
    valActs = 4
    
    # create environment object from above
    scoreTotal = 0
    for ep in range(numEpisodes):
        state = env.reset()
        scoreEp = 0
        if numEpisodes > 1:
            numRandFrames = random.randint(0,30)
            for i in range(numFrames): # frame loop
                if i < numRandFrames:
                    env.step(env.sample_action())
                    continue
                    
                # action selection
                act = agent.act(state)
                
                state, reward, isDone = env.step(act)

                #print('State: ' + str(state))
                scoreEp += reward
                
                if isDone:
                    break

            print('Agent #' + str(agent.agentNum) + 
              ' | Ep #' + str(ep) + ' | Score: ' + str(scoreEp))
    
        scoreTotal += scoreEp
    scoreTotal /= numEpisodes
    env.close()
    agent.reward(scoreTotal, envName)
    scoreList.append((agent.team.id, agent.team.outcomes))

### All Together Now!

In [26]:
acts = 4
# pAtomic equal to 1 so teams are single nodes
    # starting off, teams don't reference teams
trainer = Trainer(actions=acts, teamPopSize=10, pActAtom=0.999999999)

envName = 'GridWorldv1'
allScores = []
for gen in range(100):
    scoreList = []

    agents = trainer.getAgents()

    for agent in agents:
        runAgent([agent, envName, scoreList, 10, 1000])

    # apply scores, must do this when multiprocessing
    # because agents can't refer to trainer
    teams = trainer.applyScores(scoreList)

    trainer.evolve(tasks=[envName]) # go into next gen

    # an easier way to track stats than the above example
    scoreStats = trainer.fitnessStats

    allScores.append((scoreStats['min'], scoreStats['max'], scoreStats['average']))

    print('Gen: ' + str(gen))
    print('Results so far: ' + str(allScores))

print('Results:\nMin, Max, Avg')
for score in allScores:
    print(score[0],score[1],score[2])

Agent #0 | Ep #0 | Score: -0.99
Agent #0 | Ep #1 | Score: -0.99
Agent #0 | Ep #2 | Score: -0.94
Agent #0 | Ep #3 | Score: -0.9299999999999999
Agent #0 | Ep #4 | Score: -0.94
Agent #0 | Ep #5 | Score: -0.9299999999999999
Agent #0 | Ep #6 | Score: -0.96
Agent #0 | Ep #7 | Score: -0.96
Agent #0 | Ep #8 | Score: -0.92
Agent #0 | Ep #9 | Score: -0.92
Agent #1 | Ep #0 | Score: -0.96
Agent #1 | Ep #1 | Score: -1
Agent #1 | Ep #2 | Score: -1
Agent #1 | Ep #3 | Score: -0.98
Agent #1 | Ep #4 | Score: -1
Agent #1 | Ep #5 | Score: -0.98
Agent #1 | Ep #6 | Score: -1
Agent #1 | Ep #7 | Score: -0.99
Agent #1 | Ep #8 | Score: -0.98
Agent #1 | Ep #9 | Score: -1
Agent #2 | Ep #0 | Score: -0.9
Agent #2 | Ep #1 | Score: -0.92
Agent #2 | Ep #2 | Score: -0.92
Agent #2 | Ep #3 | Score: -0.91
Agent #2 | Ep #4 | Score: -0.94
Agent #2 | Ep #5 | Score: -0.92
Agent #2 | Ep #6 | Score: -1
Agent #2 | Ep #7 | Score: -0.95
Agent #2 | Ep #8 | Score: -0.9299999999999999
Agent #2 | Ep #9 | Score: -0.91
Agent #3 | Ep #0 

Adding 7e5939bc-34ee-4238-b4e2-00b58492193b to trainer learners
Adding 9ae5573a-7011-471b-a17d-9c4510999260 to trainer learners
Adding 399faa1e-1c40-4472-92fb-289e5553b43e to trainer learners
Adding df44219f-da48-4b69-b33b-c6bfd1b90664 to trainer learners
Adding b7244b78-1705-4711-9eb5-9b417f7a58d8 to trainer learners
Adding 7464a413-5d28-416d-aff0-32f272c8cce9 to trainer learners
Adding 605684b9-2fc1-463e-a368-505310f90df8 to trainer learners
Adding 34292d91-a77f-4538-aa32-b4577c9be79d to trainer learners
Adding 8c1423b2-dc8f-43ef-b98c-21d5ec694ecb to trainer learners
Adding fd25eddf-d0be-4590-97af-fa96eb17515a to trainer learners
Adding 057da8d7-3785-49ff-a87d-224f30f336a6 to trainer learners
Adding 96b6b664-18ee-42ed-92de-4a2babc8d44c to trainer learners
Adding ed41b89b-5b7b-4164-aa17-7d859f667f8a to trainer learners
Adding 166a4559-94de-4167-b928-b892e016208e to trainer learners
Gen: 7
Results so far: [(-0.993, 19.258, 1.70229999999999), (-0.9630000000000001, 69.77099999999999, 8.7

Agent #7 | Ep #5 | Score: -0.99
Agent #7 | Ep #6 | Score: -0.95
Agent #7 | Ep #7 | Score: -1
Agent #7 | Ep #8 | Score: -1
Agent #7 | Ep #9 | Score: -0.99
Agent #8 | Ep #0 | Score: -0.92
Agent #8 | Ep #1 | Score: 100.07000000000001
Agent #8 | Ep #2 | Score: 100.06
Agent #8 | Ep #3 | Score: -0.98
Agent #8 | Ep #4 | Score: -0.94
Agent #8 | Ep #5 | Score: -0.9299999999999999
Agent #8 | Ep #6 | Score: -0.92
Agent #8 | Ep #7 | Score: 100.08
Agent #8 | Ep #8 | Score: -0.92
Agent #8 | Ep #9 | Score: 100.09
Agent #9 | Ep #0 | Score: 100.08
Agent #9 | Ep #1 | Score: -0.95
Agent #9 | Ep #2 | Score: -0.9299999999999999
Agent #9 | Ep #3 | Score: -0.92
Agent #9 | Ep #4 | Score: -0.9299999999999999
Agent #9 | Ep #5 | Score: 100.05000000000001
Agent #9 | Ep #6 | Score: 100.08
Agent #9 | Ep #7 | Score: 100.06
Agent #9 | Ep #8 | Score: -0.94
Agent #9 | Ep #9 | Score: 100.08
Adding dd0c7b02-95ab-4620-b4fa-174fb097db63 to trainer learners
Adding 92f27ea3-6a10-4502-88ba-27a6e8d1516a to trainer learners
Add

Agent #7 | Ep #2 | Score: -0.94
Agent #7 | Ep #3 | Score: -0.95
Agent #7 | Ep #4 | Score: 100.05000000000001
Agent #7 | Ep #5 | Score: -0.92
Agent #7 | Ep #6 | Score: -0.9299999999999999
Agent #7 | Ep #7 | Score: 100.09
Agent #7 | Ep #8 | Score: 100.09
Agent #7 | Ep #9 | Score: -0.95
Agent #8 | Ep #0 | Score: -0.92
Agent #8 | Ep #1 | Score: -0.92
Agent #8 | Ep #2 | Score: 100.06
Agent #8 | Ep #3 | Score: -0.92
Agent #8 | Ep #4 | Score: -0.92
Agent #8 | Ep #5 | Score: 100.09
Agent #8 | Ep #6 | Score: 100.04
Agent #8 | Ep #7 | Score: -0.9299999999999999
Agent #8 | Ep #8 | Score: 100.07000000000001
Agent #8 | Ep #9 | Score: 100.08
Agent #9 | Ep #0 | Score: 100.09
Agent #9 | Ep #1 | Score: 100.09
Agent #9 | Ep #2 | Score: 100.08
Agent #9 | Ep #3 | Score: -0.9299999999999999
Agent #9 | Ep #4 | Score: -0.95
Agent #9 | Ep #5 | Score: -0.9299999999999999
Agent #9 | Ep #6 | Score: 100.08
Agent #9 | Ep #7 | Score: 100.08
Agent #9 | Ep #8 | Score: 100.08
Agent #9 | Ep #9 | Score: 100.08
Adding a1

Agent #9 | Ep #5 | Score: -0.94
Agent #9 | Ep #6 | Score: -0.96
Agent #9 | Ep #7 | Score: 100.06
Agent #9 | Ep #8 | Score: -0.9299999999999999
Agent #9 | Ep #9 | Score: 100.05000000000001
Adding a61ba329-5eed-4403-887d-4ea96e7c854f to trainer learners
Adding ee79f0cd-d2bf-4de5-9274-1fccf7bd9159 to trainer learners
Adding f6a61e67-95e7-4917-ba35-0516ab6a214c to trainer learners
Adding f71628d7-4cde-4df2-9c81-72213059e5fc to trainer learners
Adding e2598c88-0588-4a1a-8056-b3ea22903e32 to trainer learners
Adding ea46c7eb-46cb-41bf-a947-4d091cd6e6de to trainer learners
Adding b9415873-9947-4911-bb8d-6f1d8388360f to trainer learners
Adding 0fc06511-86ce-4c15-bb82-10953c074053 to trainer learners
Adding 89d5705f-c390-4f8e-ac8b-624cf23970d8 to trainer learners
Adding 86d6c478-315b-4092-95d3-0853b95c7214 to trainer learners
Adding b17ca184-1606-4b27-9ae8-647969fc06a1 to trainer learners
Adding 409cf093-c75d-4539-8136-a0d935de5c76 to trainer learners
Adding a49711a8-9b3d-437d-8b3d-722a03d8cc15 

...

Agent #7 | Ep #6 | Score: -0.96
Agent #7 | Ep #7 | Score: -1
Agent #7 | Ep #8 | Score: -1
Agent #7 | Ep #9 | Score: -1
Agent #8 | Ep #0 | Score: 100.06
Agent #8 | Ep #1 | Score: 100.09
Agent #8 | Ep #2 | Score: -0.92
Agent #8 | Ep #3 | Score: 100.09
Agent #8 | Ep #4 | Score: 100.08
Agent #8 | Ep #5 | Score: -0.92
Agent #8 | Ep #6 | Score: 100.06
Agent #8 | Ep #7 | Score: -0.92
Agent #8 | Ep #8 | Score: 100.07000000000001
Agent #8 | Ep #9 | Score: -0.94
Agent #9 | Ep #0 | Score: -0.92
Agent #9 | Ep #1 | Score: 100.08
Agent #9 | Ep #2 | Score: -0.92
Agent #9 | Ep #3 | Score: -0.95
Agent #9 | Ep #4 | Score: -0.92
Agent #9 | Ep #5 | Score: 100.08
Agent #9 | Ep #6 | Score: 100.06
Agent #9 | Ep #7 | Score: -0.92
Agent #9 | Ep #8 | Score: 100.08
Agent #9 | Ep #9 | Score: -0.9299999999999999
Adding b093e878-a0de-4382-8c54-23de1ac6fbee to trainer learners
Adding f750bfe6-c3ae-477e-9b8b-aae0238be04f to trainer learners
Adding 393fa7bc-a195-4f91-8fb7-34bfc836b4be to trainer learners
Adding bc0320

Agent #7 | Ep #0 | Score: -0.96
Agent #7 | Ep #1 | Score: -0.92
Agent #7 | Ep #2 | Score: 100.09
Agent #7 | Ep #3 | Score: -0.94
Agent #7 | Ep #4 | Score: -1
Agent #7 | Ep #5 | Score: -0.92
Agent #7 | Ep #6 | Score: -0.96
Agent #7 | Ep #7 | Score: -0.9299999999999999
Agent #7 | Ep #8 | Score: -0.97
Agent #7 | Ep #9 | Score: -0.97
Agent #8 | Ep #0 | Score: -0.94
Agent #8 | Ep #1 | Score: -0.97
Agent #8 | Ep #2 | Score: -0.94
Agent #8 | Ep #3 | Score: 100.08
Agent #8 | Ep #4 | Score: 100.09
Agent #8 | Ep #5 | Score: -0.9299999999999999
Agent #8 | Ep #6 | Score: 100.07000000000001
Agent #8 | Ep #7 | Score: 100.07000000000001
Agent #8 | Ep #8 | Score: -0.92
Agent #8 | Ep #9 | Score: 100.06
Agent #9 | Ep #0 | Score: -0.9299999999999999
Agent #9 | Ep #1 | Score: -0.9299999999999999
Agent #9 | Ep #2 | Score: 100.07000000000001
Agent #9 | Ep #3 | Score: -0.98
Agent #9 | Ep #4 | Score: 100.05000000000001
Agent #9 | Ep #5 | Score: -0.92
Agent #9 | Ep #6 | Score: 100.09
Agent #9 | Ep #7 | Score: 

Adding 7ef82f51-ccc9-4fe2-9c3b-3bab81097696 to trainer learners
Adding 1ebd082f-e573-4c0e-bf4f-92dc607b4172 to trainer learners
Adding 42f35a92-e162-43d1-bb79-e4696c098ece to trainer learners
Adding 799b0e91-f66e-4a7a-ad5b-1a08918044ba to trainer learners
Adding e9f2cf17-ba86-46b7-a86e-0561f6faf08a to trainer learners
Adding 9c14dd0f-c4bc-447c-9547-0bda7c07919a to trainer learners
Adding aedcd277-1605-4821-98ed-49b0c327d33b to trainer learners
Gen: 84
Results so far: [(-0.993, 19.258, 1.70229999999999), (-0.9630000000000001, 69.77099999999999, 8.799599999999987), (-0.93, 69.77099999999999, 19.766299999999976), (-0.9870000000000001, 69.77099999999999, 24.308200000000003), (-0.993, 69.77099999999999, 27.3368), (-0.9730000000000001, 69.77099999999999, 27.3339), (-0.944, 69.77099999999999, 40.4753), (-0.924, 69.77099999999999, 45.5294), (-0.985, 69.77099999999999, 48.5518), (-0.991, 69.77099999999999, 45.5178), (-0.992, 69.77099999999999, 47.544999999999995), (-0.9369999999999999, 69.77099

Agent #7 | Ep #6 | Score: -0.98
Agent #7 | Ep #7 | Score: -0.99
Agent #7 | Ep #8 | Score: -1
Agent #7 | Ep #9 | Score: -0.99
Agent #8 | Ep #0 | Score: -1
Agent #8 | Ep #1 | Score: -0.98
Agent #8 | Ep #2 | Score: -1
Agent #8 | Ep #3 | Score: -0.95
Agent #8 | Ep #4 | Score: -0.97
Agent #8 | Ep #5 | Score: -0.99
Agent #8 | Ep #6 | Score: -0.98
Agent #8 | Ep #7 | Score: -0.99
Agent #8 | Ep #8 | Score: -0.97
Agent #8 | Ep #9 | Score: -0.99
Agent #9 | Ep #0 | Score: 100.09
Agent #9 | Ep #1 | Score: -0.96
Agent #9 | Ep #2 | Score: -0.95
Agent #9 | Ep #3 | Score: -0.96
Agent #9 | Ep #4 | Score: 100.06
Agent #9 | Ep #5 | Score: -0.94
Agent #9 | Ep #6 | Score: -0.92
Agent #9 | Ep #7 | Score: -0.92
Agent #9 | Ep #8 | Score: -0.92
Agent #9 | Ep #9 | Score: 100.09
Adding 253f294e-283c-4567-ac96-e9d25a0ce7b9 to trainer learners
Adding a97f0da4-a341-42b2-8f14-f6a05214f3de to trainer learners
Adding db01e6a1-b7ed-4e04-9d33-62c2cde466a9 to trainer learners
Adding aee5f48c-7179-402f-ab21-754707e5e413 to

Agent #6 | Ep #6 | Score: 100.08
Agent #6 | Ep #7 | Score: -0.94
Agent #6 | Ep #8 | Score: -0.94
Agent #6 | Ep #9 | Score: 100.08
Agent #7 | Ep #0 | Score: -0.9
Agent #7 | Ep #1 | Score: -0.9299999999999999
Agent #7 | Ep #2 | Score: -0.91
Agent #7 | Ep #3 | Score: -0.91
Agent #7 | Ep #4 | Score: -0.9299999999999999
Agent #7 | Ep #5 | Score: -0.9299999999999999
Agent #7 | Ep #6 | Score: -0.94
Agent #7 | Ep #7 | Score: -0.97
Agent #7 | Ep #8 | Score: -0.91
Agent #7 | Ep #9 | Score: -0.92
Agent #8 | Ep #0 | Score: 100.06
Agent #8 | Ep #1 | Score: -0.97
Agent #8 | Ep #2 | Score: 100.09
Agent #8 | Ep #3 | Score: 100.09
Agent #8 | Ep #4 | Score: -0.97
Agent #8 | Ep #5 | Score: 100.03
Agent #8 | Ep #6 | Score: -0.9299999999999999
Agent #8 | Ep #7 | Score: -0.92
Agent #8 | Ep #8 | Score: -0.9299999999999999
Agent #8 | Ep #9 | Score: -0.92
Agent #9 | Ep #0 | Score: -0.9
Agent #9 | Ep #1 | Score: 100.09
Agent #9 | Ep #2 | Score: -0.94
Agent #9 | Ep #3 | Score: 100.07000000000001
Agent #9 | Ep #4